In [1]:
1+2

3

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, TrainerCallback
from datasets import load_dataset
from peft import LoraConfig, get_peft_model


In [ ]:
print(torch.__version__)
print(torchvision.__version__)

In [3]:
print(torch.cuda.is_available()) 

True


> Dataset explaining Neuromorphic computing.

In [4]:
#Dataset - Primary Students-Teacher interactions
from datasets import load_dataset

dataset = load_dataset("ajibawa-2023/Education-Young-Children")

README.md:   0%|          | 0.00/124 [00:00<?, ?B/s]

Education-Young-Children-0-Final.json:   0%|          | 0.00/294M [00:00<?, ?B/s]

Education-Young-Children-1-Final.json:   0%|          | 0.00/292M [00:00<?, ?B/s]

Education-Young-Children-2-Final.json:   0%|          | 0.00/294M [00:00<?, ?B/s]

Education-Young-Children-3-Final.json:   0%|          | 0.00/392M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/255848 [00:00<?, ? examples/s]

In [5]:
print(ds["train"][0])

{'prompt': 'Write a long and very detailed course unit for a textbook on "Fourth-Year Japanese, Second Quarter" intended for young children.\nWe have already covered chapter(s) "1. Introduction", "2. Structure of Japanese", "3. Writings in different genres and styles", "4. Using knowledge in writing", "5. Expressing opinions on a variety of topics", "6. Original writings" and are now writing a chapter on "7. Conclusion".\nWe have already covered the following sub-units in the current chapter: "7.1. Review of course content".\nWrite the new sub-unit titled "7.2. Future directions for Japanese language learning" while trying to be:\n- Rigorous - you create challenging textbooks that cover the material in depth.\n- Engaging - your textbooks have a narrative arc and engaging tone, like the writing of Michael Lewis.\n- Applied - you use specific and practical examples. For example, if the topic is integration in calculus, include equations and proofs of the concept you\'re teaching. As anot

In [103]:
#v2
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [104]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

In [105]:
peft_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [101]:
def tokenize_function(examples):
    prompt_template = "<|im_start|>user\n{}<|im_end|>\n<|im_start|>assistant\n"
    formatted_prompts = [prompt_template.format(p) for p in examples["prompt"]]
    formatted_texts = [t + "<|im_end|>" for t in examples["text"]]
    
    # Tokenize inputs (prompts + responses for causal LM)
    tokenized_inputs = tokenizer(
        formatted_prompts,
        formatted_texts,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    
    # Create proper labels (for causal LM, labels are the same as input_ids)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()
    
    return tokenized_inputs

In [102]:
# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Removing columns that aren't needed for training
columns_to_remove = [col for col in tokenized_datasets.column_names["train"] 
                    if col not in ["input_ids", "attention_mask", "labels"]]
tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)

Map:   0%|          | 0/255848 [00:00<?, ? examples/s]

In [106]:
for col in tokenized_datasets.column_names["train"]:
    print(col)

input_ids
attention_mask
labels


In [111]:
training_args = TrainingArguments(
    fp16=True,
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=2,  
    learning_rate=2e-5,  
    warmup_steps=100,
    save_strategy="epoch",
    logging_dir="./logs",
    run_name="padh.ai_FT",
    report_to="none", 
    save_steps=5000,  
    logging_steps=200,
)

class DebugCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        print(logs) 
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    callbacks=[DebugCallback()] 
)


In [112]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [113]:
trainer.train()

Step,Training Loss
200,1.864500
400,0.989100
600,0.890600
800,0.855300
1000,0.845700
1200,0.845600
1400,0.827800
1600,0.825000
1800,0.824700
2000,0.812200


{'loss': 1.8645, 'grad_norm': 2.46943998336792, 'learning_rate': 1.9175000000000002e-05, 'epoch': 0.16}
{'loss': 0.9891, 'grad_norm': 1.8559706211090088, 'learning_rate': 1.7516666666666668e-05, 'epoch': 0.32}
{'loss': 0.8906, 'grad_norm': 1.9598642587661743, 'learning_rate': 1.5850000000000002e-05, 'epoch': 0.48}
{'loss': 0.8553, 'grad_norm': 2.086113214492798, 'learning_rate': 1.4183333333333334e-05, 'epoch': 0.64}
{'loss': 0.8457, 'grad_norm': 1.8515973091125488, 'learning_rate': 1.2516666666666668e-05, 'epoch': 0.8}
{'loss': 0.8456, 'grad_norm': 1.826533317565918, 'learning_rate': 1.0850000000000001e-05, 'epoch': 0.96}
{'loss': 0.8278, 'grad_norm': 1.8816505670547485, 'learning_rate': 9.183333333333334e-06, 'epoch': 1.12}
{'loss': 0.825, 'grad_norm': 2.1385412216186523, 'learning_rate': 7.516666666666668e-06, 'epoch': 1.28}
{'loss': 0.8247, 'grad_norm': 2.090642213821411, 'learning_rate': 5.85e-06, 'epoch': 1.44}
{'loss': 0.8122, 'grad_norm': 1.9559381008148193, 'learning_rate': 4.

TrainOutput(global_step=2500, training_loss=0.9285867492675781, metrics={'train_runtime': 3519.8664, 'train_samples_per_second': 2.841, 'train_steps_per_second': 0.71, 'total_flos': 3.181482344448e+16, 'train_loss': 0.9285867492675781, 'epoch': 2.0})

In [115]:
ls "./tinyllama_finetuned"

adapter_config.json        special_tokens_map.json  tokenizer.model
adapter_model.safetensors  tokenizer_config.json
README.md                  tokenizer.json


In [114]:
model.save_pretrained("./tinyllama_finetuned")
tokenizer.save_pretrained("./tinyllama_finetuned")

('./tinyllama_finetuned/tokenizer_config.json',
 './tinyllama_finetuned/special_tokens_map.json',
 './tinyllama_finetuned/tokenizer.model',
 './tinyllama_finetuned/added_tokens.json',
 './tinyllama_finetuned/tokenizer.json')

In [116]:
def generate_text(prompt):
    # Format the prompt according to the model's expected format
    formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
    
    output = model.generate(
        **inputs, 
        max_length=512, 
        temperature=0.7, 
        top_p=0.9, 
        do_sample=True,
        repetition_penalty=1.2
    )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=False)
    # Extract just the assistant's response
    assistant_response = generated_text.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0]
    return assistant_response

In [124]:
prompt = input('What is on your mind?')
print(generate_text(prompt))

What is on your mind? explain hyperloop to a kid


Introduction: Hyperloops, or Hyper-Velocity Transportation Systems (HVTS), are high-speed transport systems that travel at incredibly fast speeds through empty space. These futuristic vehicles are powered by advanced technology and can move people, goods, and even entire cargo containers between locations in seconds. However, not everyone understands the concept of hyperloops due to their complexity and technical nature. In this guide for parents who want to explain what hyperloops are all about to their children, we will break down everything you need to know about them in simple terms.

Hyperloops are Fast, Fun, and Exciting!

Imagine being able to hop onboard a ride like a roller coaster but without having to worry about slowing down when entering or exiting stations. That's what makes hyperloops so amazing! By using magnetic fields, these vehicles propel themselves upwards into the sky while travelling incredible distances with minimal effort from passengers. This means they save t

In [126]:
ls "/kaggle/working/fine_tuned_model"

adapter_config.json        special_tokens_map.json  tokenizer.model
adapter_model.safetensors  tokenizer_config.json
README.md                  tokenizer.json


In [130]:
!zip -r fine_tuned_model.zip fine_tuned_model

  adding: fine_tuned_model/ (stored 0%)
  adding: fine_tuned_model/adapter_config.json (deflated 53%)
  adding: fine_tuned_model/special_tokens_map.json (deflated 79%)
  adding: fine_tuned_model/tokenizer.model (deflated 55%)
  adding: fine_tuned_model/adapter_model.safetensors (deflated 8%)
  adding: fine_tuned_model/README.md (deflated 66%)
  adding: fine_tuned_model/tokenizer_config.json (deflated 68%)
  adding: fine_tuned_model/tokenizer.json (deflated 85%)
